In [ ]:
from huggingface_hub import login
import wandb
login(token = "YOUR_KEY")
wandb.login(key = "YOUR_KEY")
run = wandb.init(
    project='Fine-tune gemma-3n-e4b',
    job_type="training",
    anonymous="allow"
)

In [ ]:
csv_file_name = "Datasets-note-summary.csv"
base_model_name = "unsloth/gemma-3n-E4B-it"
new_model_name = "gemma-3n-privnurse-note-summary-v1"

In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = base_model_name,
    dtype = None, # None for auto detection
    max_seq_length = 32768, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

In [ ]:
###########################
# Let's finetune Gemma 3N #
###########################

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 32,           # Larger = higher accuracy, but might overfit
    lora_alpha = 32,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 1205,
)

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files=csv_file_name, split="all")
dataset = dataset.shuffle(seed=56)
dataset[100]

In [ ]:
# System message for the assistant 
system_prompt = """<Role>
    You are an expert Emergency Department (ED) physician AI assistant. Your purpose is to generate a clinically accurate "Treatment Course" summary.
</Role>

<Task>
    Based on the provided XML data, generate a concise, single-paragraph summary by synthesizing the clinically significant events of the ED encounter. A complete summary should include t>
    - **Diagnosis:** The primary working diagnosis.
    - **Key Examinations:** Clinically significant lab/imaging results.
    - **Key Medications/Interventions:** Important treatments administered.
    - **Consultations:** (See special rule below).
    - **Disposition & Follow-up Plan:** The final outcome. (Always be written at the end)
</Task>

<Special Rule for Consultations>
- **IF** a consultation occurred (e.g., OBYN, NEUR, CRS), the summary **MUST** mention it detailly and its outcome.
- **IF NO** consultation is mentioned in the XML, the summary **MUST NOT** mention one.
</Special Rule for Consultations>

<CorePrinciple priority="high">
    **Factual Grounding is Mandatory:** The generated summary must be **strictly and exclusively** based on the information within the provided <PatientEncounter> XML. **Do not hallucinat>
</CorePrinciple>

<OutputFormat>
    The output must be only the summary text. Do not include any other commentary. DO NOT Thought.
</OutputFormat>
"""

In [ ]:
def formatting_prompts_func(examples):
    # 構建 conversations 列表，每個元素包含 system, user, assistant 的訊息
    convos = []
    for i in range(len(examples["input_text"])):
        convo = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": examples["input_text"][i]},
            {"role": "assistant", "content": examples["distilled_output"][i]}
        ]
        convos.append(convo)
    
    # 應用 chat template 並移除 <bos> 前綴
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False).removeprefix('<bos>') for convo in convos]
    
    return {"text": texts}

# 使用批次處理
dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
dataset[100]["text"]

In [ ]:
###################
# Train the model #
###################

In [ ]:
from trl import SFTTrainer, SFTConfig
training_arguments = SFTConfig(
    dataset_text_field="text",
    output_dir=new_model_name,
    per_device_train_batch_size=1,     # 預設2
    gradient_accumulation_steps=16,     # 預設8
    optim="adamw_torch_fused",         # Options: adamw_hf, adamw_torch, adamw_torch_fused, adamw_8bit
    num_train_epochs=1,                # 建議不高於6；設定到7(含)以上會出現overfitting
    ###############
    # per_device_eval_batch_size=2,    # 預設2
    # evaluation_strategy="steps",
    eval_strategy='no',
    # eval_steps=10,
    ###############
    lr_scheduler_type = "linear",      # Options: cosine(when epoch>=6), constant(when epoch<6), linear
    max_grad_norm=0.3,                 # Default: 0.3
    warmup_ratio=0.1,                 # Default: 0.03
    # warmup_steps=30,                 # Default: 30
    learning_rate=2e-4,                # Default: 2e-4
    # weight_decay=0.01,               # [new]Default: 0
    # adam_beta1=0.9,                  # [new]Default: 0.9
    # adam_beta2=0.95,                 # [new]Default: 0.999
    # seed=1205,
    logging_strategy="steps",
    logging_steps=10,                # When batch size is big, logging steps should be reduced to 2-5 not 10
    save_strategy="epoch",
    fp16=False,
    bf16=True,
    # group_by_length=True,
    report_to="wandb",
    max_seq_length=32768,
    seed = 1205,
    # packing= False,
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = training_arguments,
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:
# [Bug]: InductorError: RuntimeError: Failed to run autotuning code block: PY_SSIZE_T_CLEAN macro must be defined for '#' formats
# Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"
# [Solution]: Disable TorchInductor Compilation. Add this before your inference code:
import torch
torch._dynamo.config.disable = True
# Or alternatively, disable specific optimizations
# torch._dynamo.config.suppress_errors = True
# >> The error is related to PyTorch's aggressive optimization trying to compile the vision components of Gemma 3, which can be disabled without affecting functionality.)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
wandb.finish()

In [ ]:
#############
# Inference #
#############

In [ ]:
# [Bug]: InductorError: RuntimeError: Failed to run autotuning code block: PY_SSIZE_T_CLEAN macro must be defined for '#' formats
# Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"
# [Solution]: Disable TorchInductor Compilation. Add this before your inference code:
import torch
torch._dynamo.config.disable = True
# Or alternatively, disable specific optimizations
# torch._dynamo.config.suppress_errors = True
# >> The error is related to PyTorch's aggressive optimization trying to compile the vision components of Gemma 3, which can be disabled without affecting functionality.)

In [ ]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

def process_consultation(model, tokenizer, consultation_content, temperature=0.1, max_tokens=2048):
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = "gemma-3",
    )
    messages = [{"role": "system",  "content": [{"type": "text", "text": system_prompt}]},
                {"role": "user", "content": [{"type": "text", "text": consultation_content}]}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
        tokenize = True,
        return_dict = True,
    ).to("cuda")

    _ = model.generate(
        **inputs,
        max_new_tokens = max_tokens,
        temperature = temperature,
        ##############################################################################
        # temperature = 1.0, top_p = 0.95, top_k = 64, # Recommended Gemma-3 settings!
        ##############################################################################
        top_p = 0.95,
        top_k = 64,
        ##############################################################################
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

In [ ]:
consultation_content = """<PatientEncounter summary_length_style="long">
    <Summary>
        <PrimaryDiagnosis>DM 2 w hyperglycemia-(HbA1c> 14.);</PrimaryDiagnosis>
<ChiefComplaint>Numbness below knees and both hands for half year. Transfer from NEPH OPD for DM control .
</ChiefComplaint>
<PresentIllness>A case of DM.
&nbsp;Treadmill&#39;s ECG(4/30):negative finding.&nbsp; heart echogram(5/07) were done in CV OPD.
&nbsp;
According to the statement from the patient&nbsp; , there is no trauma, vomiting,diarrhea, fever,chillness, chest or abdominal/back pain,hematuria,urine frequency or urgency, cough,sore throat,conscious change,dizziness, headache, focal limb weakness or numbness,seizure, SOB,DOE, dyspnea, tarry or bloody stool,<br />
&nbsp;
&nbsp;
&nbsp;
</PresentIllness>
    </Summary>
    <ChronologicalEvents>
        <LabReportGroup date="2024-05-08">
<Item name="C reactive protein [Mass/volume] in Serum or Plasma by High sensitivity method">1.071</Item>
<Item name="Systolic blood pressure">149</Item>
<Item name="Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)">121.51</Item>
<Item name="Potassium [Moles/volume] in Serum or Plasma">4.7</Item>
<Item name="Sodium [Moles/volume] in Serum or Plasma">134</Item>
<Item name="Lymphocytes [#/volume] in Blood">1091</Item>
<Item name="Neutrophils [#/volume] in Blood">7829</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">13.5</Item>
<Item name="MCH [Entitic mass] by Automated count">31.8</Item>
<Item name="MCHC [Mass/volume] by Automated count">32.7</Item>
<Item name="Platelet mean volume [Entitic volume] in Blood by Automated count">9.2</Item>
<Item name="Erythrocyte distribution width [Ratio] by Automated count">12.9</Item>
<Item name="Leukocytes [#/volume] in Blood">9490</Item>
<Item name="Diastolic blood pressure">66</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">13.5</Item>
<Item name="Leukocytes [#/volume] in Blood">9490</Item>
<Item name="Creatinine [Mass/volume] in Serum or Plasma">0.66</Item>
<Item name="Platelets [#/volume] in Blood by Automated count">237</Item>
<Item name="Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)">121.51</Item>
<Item name="Ketones [Moles/volume] in Blood"><0.1</Item>
<Item name="Basophils/100 leukocytes in Blood">0.2</Item>
<Item name="Eosinophils/100 leukocytes in Blood">0.7</Item>
<Item name="Lymphocytes/100 leukocytes in Blood">11.5</Item>
<Item name="Monocytes/100 leukocytes in Blood">5.1</Item>
<Item name="Neutrophils/100 leukocytes in Blood">82.5</Item>
<Item name="Nucleated erythrocytes/100 leukocytes [Ratio] in Blood">0.0</Item>
<Item name="Platelets [#/volume] in Blood by Automated count">237</Item>
<Item name="aPTT in Platelet poor plasma by Coagulation assay">26.2</Item>
<Item name="Bilirubin.total [Presence] in Urine by Automated test strip">Negative</Item>
<Item name="Clarity in Urine by Refractometry automated">Clear</Item>
<Item name="Color of Urine by Auto">Yellow</Item>
<Item name="Glucose [Mass/volume] in Urine by Automated test strip">>=1000</Item>
<Item name="Ketones [Presence] in Urine by Automated test strip">Negative</Item>
<Item name="Leukocyte esterase [Presence] in Urine by Automated test strip">Negative</Item>
<Item name="Nitrite [Presence] in Urine by Automated test strip">Negative</Item>
<Item name="Hemoglobin [Presence] in Urine by Automated test strip">Negative</Item>
<Item name="pH of Urine by Automated test strip">7.0</Item>
<Item name="Protein [Presence] in Urine by Automated test strip">1+</Item>
<Item name="Specific gravity of Urine by Refractometry automated">1.036</Item>
<Item name="Urobilinogen [Units/volume] in Urine by Test strip">0.2</Item>
<Item name="Creatine kinase [Enzymatic activity/volume] in Serum or Plasma">84</Item>
<Item name="Creatine kinase [Enzymatic activity/volume] in Serum or Plasma">85</Item>
<Item name="Troponin T.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method">18.69</Item>
<Item name="Albumin [Mass/volume] in Serum or Plasma">3.9</Item>
<Item name="Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma by No addition of P-5'-P">20</Item>
<Item name="Glucose [Mass/volume] in Serum or Plasma">495</Item>
<Item name="Epithelial cells [#/area] in Urine sediment by Microscopy high power field">0-5</Item>
<Item name="Erythrocytes [#/area] in Urine sediment by Microscopy high power field">0-2</Item>
<Item name="Leukocytes [#/area] in Urine sediment by Microscopy high power field">0-5</Item>
<Item name="Natriuretic peptide.B prohormone N-Terminal [Mass/volume] in Serum or Plasma by Immunoassay">109.2</Item>
<Item name="Prothrombin time (PT) in Control Platelet poor plasma by Coagulation assay">10.3</Item>
<Item name="Prothrombin time (PT)">9.4</Item>
<Item name="INR in Platelet poor plasma by Coagulation assay">0.91</Item>
<Item name="Bilirubin.total [Mass/volume] in Serum or Plasma">0.46</Item>
<Item name="Troponin T.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method">16.96</Item>
<Item name="Base excess in Venous blood by calculation">3.3</Item>
<Item name="Bicarbonate [Moles/volume] in Venous blood">30.5</Item>
<Item name="Carbon dioxide [Partial pressure] in Venous blood">57.7</Item>
<Item name="pH of Venous blood">7.341</Item>
<Item name="Oxygen [Partial pressure] in Venous blood">32.8</Item>
<Item name="Oxygen saturation in Venous blood">57.8</Item>
<Item name="Carbon dioxide, total [Moles/volume] in Venous blood">27.7</Item>
<Item name="Urea nitrogen [Mass/volume] in Serum or Plasma">16</Item>
<Item name="Creatinine [Mass/volume] in Serum or Plasma">0.66</Item>
</LabReportGroup>
<NursingEvent timestamp="2024-05-08 09:00:00">
<VitalSign type="STOOL.STOOL" value="排便:0 次" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 16:00:00">
<SOAPNote>
<Intervention>醫師診視後，依醫囑設立靜脈輸液管路、抽血檢驗，設床ED1-161後臥床休息。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-05-08 16:00:00">
<SOAPNote>
<NarrativeNote>意識清醒，室氧使用下呼吸平穩，周邊靜脈點滴滴注順暢，注射處無紅腫滲液，皮膚完整沒有壓傷，雙腳小腿麻木感存，雙足背水腫1+，獨自一人。</NarrativeNote>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-05-08 16:02:00">
<VitalSign type="BT.ORAL" value="口溫:36.0℃" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 16:02:00">
<VitalSign type="PULSE.USUAL" value="脈搏:97.0次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 16:02:00">
<VitalSign type="RESPIRATORY" value="呼吸:18.0次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 16:02:00">
<VitalSign type="BP" value="血壓:158.0/68.0mmHg" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 17:00:00">
<SOAPNote>
<Objective>GLU-Random:  495</Objective><Intervention>依醫囑予Actrapid HM inj 1U (8 U) IVD STAT</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-05-08 20:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.3 ℃" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 20:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:88 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 20:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 20:00:00">
<VitalSign type="BP" value="血壓:149/66 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 21:00:00">
<VitalSign type="GLUCOSE" value="血糖:514.0 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 21:14:00">
<VitalSign type="IS.4A" value="胰島素:部位 :左腹4A 藥物處置:Actrapid HM inj 1U 10 U SC STAT。" />
</NursingEvent>
<NursingEvent timestamp="2024-05-08 23:00:00">
<VitalSign type="GLUCOSE" value="血糖:436.0 mg/dl" />
</NursingEvent>
<LabReportGroup date="2024-05-09">
<Item name="Systolic blood pressure">133</Item>
<Item name="Bacteria identified in Urine by Culture">Gram positive coccus</Item>
<Item name="Diastolic blood pressure">63</Item>
<Item name="Oxygen saturation in Blood">96</Item>
<Item name="Thyroxine (T4) free [Mass/volume] in Serum or Plasma by Immunoassay">1.16</Item>
<Item name="Chloride [Moles/volume] in Serum or Plasma">106</Item>
<Item name="Potassium [Moles/volume] in Serum or Plasma">3.5</Item>
<Item name="Sodium [Moles/volume] in Serum or Plasma">140</Item>
<Item name="Thyrotropin [Units/volume] in Serum or Plasma by Detection limit <= 0.05 mIU/L">1.651</Item>
<Item name="Bacteria identified in Urine by Culture">Gram positive coccus</Item>
<Item name="Colony count [#] in Urine by Visual count">20000</Item>
</LabReportGroup>
<NursingEvent timestamp="2024-05-09 01:00:00">
<SOAPNote>
<NarrativeNote>意識清楚，呼吸平順，周邊靜脈留置針存，無紅腫滲液，滴注順暢，無壓力性損傷皮膚，可自解尿液，雙側床欄拉起，現臥床休息中，獨自一人生活可自理。</NarrativeNote>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-05-09 02:00:00">
<SOAPNote>
<Objective>血糖:311mg/dL</Objective><Intervention>告知醫師，依醫囑予Actrapid HM (6 U) IV STAT，續觀察血糖變化。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-05-09 02:08:00">
<VitalSign type="GLUCOSE" value="血糖:311 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2024-05-09 05:58:00">
<VitalSign type="GLUCOSE" value="血糖:135 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2024-05-09 09:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.6 ℃" />
</NursingEvent>
<NursingEvent timestamp="2024-05-09 09:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:90 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-05-09 09:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-05-09 09:00:00">
<VitalSign type="BP" value="血壓:133/63 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2024-05-09 09:00:00">
<VitalSign type="SPO2" value="SPO2:96.0 %" />
</NursingEvent>
<NursingEvent timestamp="2024-05-09 10:00:00">
<VitalSign type="GLUCOSE" value="血糖:169.0 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2024-05-09 10:00:00">
<SOAPNote>
<NarrativeNote>轉出摘要：目前問題（Situation）：目前病人因DM 2 w hyperglycemia，經醫療團隊評估轉至：62-053病房，轉出時間：113年5月9日，繼續接受治療。背景資料（Background）：糖尿病病史，門診血液報告:HbA1C >14 %，從腎臟科門診轉至急診，控制血糖。評估（Assessment）：血液報告:hsCRP: 1.071 mg/dL、Creatinine 0.66 mg/dL；意識清楚，呼吸平順，周邊靜脈留置針存，無紅腫滲液，滴注順暢，無壓力性損傷皮膚，可自解尿液，雙側床欄拉起，現臥床休息中，獨自一人生活可自理。建議（Recommendation）：注意生命徵象變化，維持管路通暢、血糖變化。</NarrativeNote>
</SOAPNote>
</NursingEvent>
    </ChronologicalEvents>
</PatientEncounter>"""
process_consultation(model, tokenizer, consultation_content, temperature=0.1, max_tokens=1024)
############# Sample Output #############
# At triage, his vital signs were stable. PE showed clear consciousness with G4V5M6,&nbsp;bilateral 
# lower distal extremities pitting edema 1+;&nbsp;symmetric breathing pattern, clear breathing sound, regular heartbeat 
# without murmur, no abdominal tenderness, no muscle guarding, no rebound pain, Murphy sign(-), Mcburney point(-), 
# Rovsiing sign(-), flank knocking pain(-), nuchal rigidity(-). NE showed bilateral&nbsp;distal lower limbs (Below ankles)&nbsp;
# decreased light tough sensation (stocking distribution); MP(all intact); Gait(no deviation). Initial lab data showed hemogram 
# within normal range (WNR), liver function profile WNR, renal function WNR, Fasting&nbsp;glucose level 495, ketone level 
# &lt;0.1, VBG no acidosi.&nbsp;CXR showed insignificant finding. The tentative diagnosis is DM peripheral neuropathy. After 
# Meta consultaion, admission for DM complication survey was suggested.

In [ ]:
consultation_content = """<PatientEncounter summary_length_style="medium">
    <Summary>
        <PrimaryDiagnosis>Syncope;Acute gastroenteritis;Diarrhea;</PrimaryDiagnosis>
<ChiefComplaint>Consious disturbance just now after diarrhea
</ChiefComplaint>
<PresentIllness>Watery diarrhea once just now, then consicousness disturbance at the bathroom, no head trauma Hx, cyanosis +/-, last for 15 seconds, self-recovered and regained consicousness as baseline
&nbsp;
High temperature (37.x&#39;c)&nbsp;noted for 1 day
Bilateral hands (Rt&gt;Lt) numbness, could eat by himself, also for 1 day
&nbsp;
Poor appetite (still &gt; 1/2 usual amount) for 2 wks, no decreased urine output
No fever, no cough, no rhinorrhea, no nausea/vomiting<br />
&nbsp;
no known drug allergy.<br />
past surgical hx: bil eyes<br />
past med hx: asthma without medical treatment current<br />
fever/cough/COVID contact history: elder brother cough and rhinorrha 2 wks ago
past COVID infection: -1.5 years ago
&nbsp;
</PresentIllness>
    </Summary>
    <ChronologicalEvents>
        <LabReportGroup date="2024-03-16">
<Item name="Oxygen saturation in Blood">99</Item>
<Item name="Diastolic blood pressure">57</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">12.5</Item>
<Item name="Leukocytes [#/volume] in Blood">11040</Item>
<Item name="Creatinine [Mass/volume] in Serum or Plasma">0.45</Item>
<Item name="Platelets [#/volume] in Blood by Automated count">242</Item>
<Item name="Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)">281.53</Item>
<Item name="Systolic blood pressure">101</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">12.5</Item>
<Item name="Magnesium [Mass/volume] in Serum or Plasma">1.9</Item>
<Item name="Lymphocytes/100 leukocytes in Blood">12.3</Item>
<Item name="Base excess in Venous blood by calculation">-1.9</Item>
<Item name="Glucose [Mass/volume] in Serum or Plasma">97</Item>
<Item name="Eosinophils/100 leukocytes in Blood">0.5</Item>
<Item name="Calcium [Mass/volume] in Serum or Plasma">9.5</Item>
<Item name="Creatine kinase [Enzymatic activity/volume] in Serum or Plasma">120</Item>
<Item name="Hematocrit [Volume Fraction] of Blood">37.6</Item>
<Item name="MCHC [Mass/volume] by Automated count">33.2</Item>
<Item name="Carbon dioxide, total [Moles/volume] in Venous blood">21.1</Item>
<Item name="C reactive protein [Mass/volume] in Serum or Plasma by High sensitivity method">1.107</Item>
<Item name="Leukocytes [#/volume] in Blood">11040</Item>
<Item name="Phosphate [Mass/volume] in Serum or Plasma">3.8</Item>
<Item name="MCH [Entitic mass] by Automated count">28.3</Item>
<Item name="pH of Venous blood">7.368</Item>
<Item name="Urea nitrogen [Mass/volume] in Serum or Plasma">11</Item>
<Item name="Chloride [Moles/volume] in Serum or Plasma">104</Item>
<Item name="Creatinine [Mass/volume] in Serum or Plasma">0.45</Item>
<Item name="Neutrophils [#/volume] in Blood">8700</Item>
<Item name="Neutrophils/100 leukocytes in Blood">78.8</Item>
<Item name="Monocytes/100 leukocytes in Blood">8.2</Item>
<Item name="Lymphocytes [#/volume] in Blood">1358</Item>
<Item name="Sodium [Moles/volume] in Serum or Plasma">138</Item>
<Item name="Erythrocyte distribution width [Ratio] by Automated count">11.6</Item>
<Item name="Lactate [Mass/volume] in Serum or Plasma">8.5</Item>
<Item name="Erythrocytes [#/volume] in Blood by Automated count">4.41</Item>
<Item name="Carbon dioxide [Partial pressure] in Venous blood">41.4</Item>
<Item name="Basophils/100 leukocytes in Blood">0.2</Item>
<Item name="Potassium [Moles/volume] in Serum or Plasma">3.7</Item>
<Item name="MCV [Entitic volume] by Automated count">85.3</Item>
<Item name="Oxygen [Partial pressure] in Venous blood">38.9</Item>
<Item name="Oxygen saturation in Venous blood">74.8</Item>
<Item name="Platelets [#/volume] in Blood by Automated count">242</Item>
<Item name="Bicarbonate [Moles/volume] in Venous blood">23.3</Item>
<Item name="Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma by No addition of P-5'-P">13</Item>
<Item name="Nucleated erythrocytes/100 leukocytes [Ratio] in Blood">0.0</Item>
<Item name="Ammonia [Mass/volume] in Plasma">33</Item>
<Item name="Platelet mean volume [Entitic volume] in Blood by Automated count">8.4</Item>
<Item name="Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method"><3.2</Item>
<Item name="Creatine kinase.MB [Mass/volume] in Serum or Plasma by Immunoassay"><1.0</Item>
<Item name="Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)">281.53</Item>
</LabReportGroup>
<NursingEvent timestamp="2024-03-16 04:00:00">
<SOAPNote>
<Intervention>醫師診視後，依醫囑設立靜脈輸液管路、抽血檢驗，Quick Check Blood Sugar, AC STAT，0.33% G/S 1000 ML IVD QD設床PER-003後臥床休息。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-03-16 04:00:00">
<VitalSign type="GLUCOSE" value="血糖:97.0 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2024-03-16 04:00:00">
<VitalSign type="BP" value="血壓:101/57 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2024-03-16 04:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:92 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-03-16 04:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-03-16 04:00:00">
<VitalSign type="BT.ORAL" value="體溫:37.4 ℃" />
</NursingEvent>
<NursingEvent timestamp="2024-03-16 04:00:00">
<VitalSign type="SPO2" value="SPO2:99.0 %" />
</NursingEvent>
<NursingEvent timestamp="2024-03-16 05:00:00">
<SOAPNote>
<Objective>經醫師診視後允許出院</Objective><Intervention>協助辦理出院，衛教按時服藥，清淡飲食，少量多餐，需要時服用止瀉藥，門診追蹤，移除靜脈留置針及手圈，</Intervention><Evaluation>家屬可了解  衛教內容。</Evaluation>
</SOAPNote>
</NursingEvent>
    </ChronologicalEvents>
</PatientEncounter>"""
process_consultation(model, tokenizer, consultation_content, temperature=0.1, max_tokens=1024)
############# Sample Output #############
# The patient had fair activity at PER. PE showed no injected throat, coarse BS, normoactive BoS, soft abdomen, no peritoneal signs, 
# no meningeal sign, no skin rash. NE grossly normal. EKG showed no arrythmia, no QT prolong, no Brugada syndrome detected. 
# No hypoglycemia noted. Lab reported no cardiac enzyme elevation also no evidence of severe bacterial infection. 
# CXR showed no cardiomegaly. Due to AGE, after well informed the toxic signs, I prescribed  medications home and suggested OPD f/u.

In [ ]:
consultation_content = """<PatientEncounter summary_length_style="medium">
    <Summary>
        <PrimaryDiagnosis>Acute bronchitis-(s/p saldolin on 4/10 2315, combivent on 4/11 0210);Influenza-(type B, s/p rapiata);Dyspnea;COVID-19，virus identified-(CT:32);</PrimaryDiagnosis>
<ChiefComplaint>SOB just now
</ChiefComplaint>
<PresentIllness>4/9 sore throat, cough, rhinorrhea, s/p LMD
4/10 SOB since 21pm(躺下來更喘)
no fever, fair activity, fair appetite
&nbsp;
PHx:1.AR
OPHx:nil
Allergy:nil
FH: mother with asthma Hx
</PresentIllness>
    </Summary>
    <ChronologicalEvents>
        <LabReportGroup date="2024-04-10">
<Item name="Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)">214.42</Item>
<Item name="Respiratory syncytial virus RNA [Presence] in Nasopharynx by NAA with probe detection">NEGATIVE</Item>
<Item name="Oxygen saturation in Blood">96</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">13.8</Item>
<Item name="Leukocytes [#/volume] in Blood">7180</Item>
<Item name="Creatinine [Mass/volume] in Serum or Plasma">0.55</Item>
<Item name="Platelets [#/volume] in Blood by Automated count">271</Item>
<Item name="Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)">214.42</Item>
<Item name="SARS-CoV-2 (COVID-19) RNA [Presence] in Nasopharynx by NAA with probe detection">Positive (SARS-CoV-2 RNA was detected)</Item>
<Item name="SARS-CoV-2 (COVID-19) RNA [Cycle Threshold #] in Respiratory system specimen by NAA with probe detection">32</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">13.8</Item>
<Item name="Lymphocytes/100 leukocytes in Blood">19.8</Item>
<Item name="Base excess in Venous blood by calculation">1.1</Item>
<Item name="Glucose [Mass/volume] in Serum or Plasma">109</Item>
<Item name="Eosinophils/100 leukocytes in Blood">8.5</Item>
<Item name="Calcium [Mass/volume] in Serum or Plasma">9.4</Item>
<Item name="Hematocrit [Volume Fraction] of Blood">42.5</Item>
<Item name="MCHC [Mass/volume] by Automated count">32.5</Item>
<Item name="Carbon dioxide, total [Moles/volume] in Venous blood">26.1</Item>
<Item name="C reactive protein [Mass/volume] in Serum or Plasma by High sensitivity method">1.254</Item>
<Item name="Leukocytes [#/volume] in Blood">7180</Item>
<Item name="MCH [Entitic mass] by Automated count">29.0</Item>
<Item name="pH of Venous blood">7.311</Item>
<Item name="Urea nitrogen [Mass/volume] in Serum or Plasma">7</Item>
<Item name="Chloride [Moles/volume] in Serum or Plasma">108</Item>
<Item name="Creatinine [Mass/volume] in Serum or Plasma">0.55</Item>
<Item name="Neutrophils [#/volume] in Blood">4466</Item>
<Item name="Neutrophils/100 leukocytes in Blood">62.2</Item>
<Item name="Monocytes/100 leukocytes in Blood">8.8</Item>
<Item name="Lymphocytes [#/volume] in Blood">1422</Item>
<Item name="Sodium [Moles/volume] in Serum or Plasma">143</Item>
<Item name="Erythrocyte distribution width [Ratio] by Automated count">12.1</Item>
<Item name="Erythrocytes [#/volume] in Blood by Automated count">4.76</Item>
<Item name="Carbon dioxide [Partial pressure] in Venous blood">58.3</Item>
<Item name="Basophils/100 leukocytes in Blood">0.7</Item>
<Item name="Potassium [Moles/volume] in Serum or Plasma">3.9</Item>
<Item name="MCV [Entitic volume] by Automated count">89.3</Item>
<Item name="Oxygen [Partial pressure] in Venous blood">28.5</Item>
<Item name="Oxygen saturation in Venous blood">47.3</Item>
<Item name="Platelets [#/volume] in Blood by Automated count">271</Item>
<Item name="Bicarbonate [Moles/volume] in Venous blood">28.8</Item>
<Item name="Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma by No addition of P-5'-P">13</Item>
<Item name="Nucleated erythrocytes/100 leukocytes [Ratio] in Blood">0.0</Item>
<Item name="Platelet mean volume [Entitic volume] in Blood by Automated count">9.1</Item>
<Item name="Influenza virus A RNA [Presence] in Nasopharynx by NAA with probe detection">NEGATIVE</Item>
<Item name="Influenza virus B RNA [Presence] in Nasopharynx by NAA with probe detection">POSITIVE</Item>
</LabReportGroup>
<NursingEvent timestamp="2024-04-10 23:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:123 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-04-10 23:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:26 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-04-10 23:00:00">
<VitalSign type="SPO2" value="SPO2:96.0 %" />
</NursingEvent>
<NursingEvent timestamp="2024-04-10 23:00:00">
<SOAPNote>
<Objective>呼吸淺快費力。</Objective><Intervention>依醫囑給予Saldolin inh. soln 0.5 AMP IH STAT。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-04-10 23:00:00">
<SOAPNote>
<Intervention>醫師診視後，依醫囑設立靜脈輸液管路、抽血檢驗，0.33% G/S 1000 ML IVD QD，設床ED1-603後臥床休息。</Intervention>
</SOAPNote>
</NursingEvent>
<LabReportGroup date="2024-04-11">
<Item name="Oxygen saturation in Blood">92</Item>
<Item name="Bacteria identified in Blood by Culture">No bacterial growth for 7 days.</Item>
</LabReportGroup>
<NursingEvent timestamp="2024-04-11 00:00:00">
<SOAPNote>
<NarrativeNote>採檢Influenza B RT PCR POSITIVE、COVID-19: Positive (Ct： 32 )。</NarrativeNote>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-04-11 00:00:00">
<SOAPNote>
<Objective>採檢Influenza B RT PCR POSITIVE。</Objective><Intervention>依醫囑予Rapiacta 300MG IVD STAT藥物使用，與家屬填寫藥物自費同意書。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-04-11 02:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.9 ℃" />
</NursingEvent>
<NursingEvent timestamp="2024-04-11 02:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:106 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-04-11 02:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:22 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-04-11 02:00:00">
<VitalSign type="SPO2" value="SPO2:92.0 %" />
<SOAPNote>
<Intervention>依醫囑予氧氣鼻導管3L/min使用，續追蹤血氧濃度變化。</Intervention><Evaluation>使用氧氣鼻導管3L/min SPO2:97%。</Evaluation>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-04-11 02:00:00">
<SOAPNote>
<Objective>醫師聽診肺部支氣管有喘鳴音。</Objective><Intervention>依醫囑予UDV Combivent inh. 2.5mL 1VIAL INH STAT藥物使用。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-04-11 03:00:00">
<SOAPNote>
<Objective>經醫師診視後允許出院。</Objective><Intervention>協助辦理，衛教返家需觀察有無呼吸喘及呼吸有喘鳴音，若有以上症狀可返回就醫，返家需採漸進式活動避免劇烈運動及採個人防護，出院帶藥須確實服用，後續採門診追蹤。</Intervention><Evaluation>家屬可了解衛教內容。</Evaluation>
</SOAPNote>
</NursingEvent>
    </ChronologicalEvents>
</PatientEncounter>

(The following is a summary written by another non-professional physician, but it contains a lot of unfounded information. However, you can refer to his format and method to write a summary.)
PE showed bilateral decreasing breathing sound. IV set was arranged. Chest xray showed bilateral increasing infiltration. Steam inhalation with saldolin was arranged and decreasing breathing sound improved. Mild wheezing could be heard and he mentioned tachypnea became better. However, mild wheezing with saturation around 92% still noted after inhalation for 3 hours. We changed to combivent use. He had fair activity and respiratory.  We informed redflag signs to the family. Discharge was arranged with OPD f/u.
"""
process_consultation(model, tokenizer, consultation_content, temperature=0.1, max_tokens=1024)
############# Sample Output #############
# PE showed bilateral decreasing breathing sound. IV set was arranged. Chest xray showed bilateral increasing infiltration. 
# Steam inhalation with saldolin was arranged and decreasing breathing sound improved. Mild wheezing could be heard and he 
# mentioned tachypnea became better. However, mild wheezing with saturation around 92% still noted after inhalation for 3 hours. 
# We changed to combivent use. He had fair activity and respiratory.  We informed redflag signs to the family. Discharge was 
# arranged with OPD f/u.

In [ ]:
consultation_content = """<PatientEncounter summary_length_style="medium">
    <Summary>
        <PrimaryDiagnosis>End stage renal disease-(HD 2.4.6, daily U/O 500cc.);UGI bleeding-(UE(02/17): huge mass, 壓迫 duodenum 2nd portion, /w periperal ulcer, s/p APC hemostasis.);Lung abscess-(RLL. CS: conservative Tx first.);Malignant tumor of pancreas-(CT(Rad): stable as last one.);</PrimaryDiagnosis>
<SecondaryDiagnosis>Anemia-(Basline Hb:9-10->8.7->pRBC 2U;OB(Trace));Coagulopathy-(PT today  14 sec,-> FFP 4U.);Hypoalbuminemia-(Albumin 1.6,-> supplement IVD BID.);Pleural effusion-(bil.);Mass of stomach-(R/O IPMN or GIST, no tissue proof.);</SecondaryDiagnosis>
<PastMedicalHistory>Hypertension;Malignancy of pancreas-(IPMN (intraductal papillary mucinous neoplasm).);Heart failure-(111/12 2D: generalized hypokinesia of LV, EF:34%);</PastMedicalHistory>
<ChiefComplaint>Tarry stool for 2 days.
- Drowsiness noted this morning.
</ChiefComplaint>
<PresentIllness>Hx of our Nephro &amp; GI F/U. Last GI admission in 112/02:
1. Suspect intraductal papillary mucinous neoplasm (IPMN) of pancreatic head or gastric high body GIST, complicated with UGI bleeding, s/p several times EUS, showing no primary malignancy. suspect **IPMN of pancreas **<br />
- 2023/02/17 EGD: One huge mass lesion with external compression over duodenum second portion with periperal shallow ulcer, s/p APC hemostasis
&nbsp;&nbsp; [PHx]:
1. Congestive heart failure, LVEF: 34%, with generalized LV hypokinesis.
2. ESRD, HD: W246, at local HD clinic.<br />
3. T2DM controled by Novorapid 12U BID<br />
4. Hypertension.<br />
5. Right parotid abscess with face cellulitis.
6. Lung nodle (0.7cm) over LUL.
&nbsp;
** This time, there were no chills or fever, no headache or neck soreness, no chest pain or dyspnea, no cold sweating,&nbsp; no tearing torsal pain, no abdominal pain or vomiting, no dysuria or urgency,&nbsp; no slurred speech or blurred vision, no focal limbs weakness.
</PresentIllness>
    </Summary>
    <ChronologicalEvents>
        <LabReportGroup date="2023-04-10">
<Item name="C reactive protein [Mass/volume] in Serum or Plasma by High sensitivity method">19.110</Item>
<Item name="Systolic blood pressure">162</Item>
<Item name="Heart rate">一般脈搏:118 次/分,特殊治療 :輸血前</Item>
<Item name="Oral temperature">體溫:36.8 ℃,特殊治療 :輸血前</Item>
<Item name="Heart rate">一般脈搏:111 次/分,特殊治療 :輸血中</Item>
<Item name="Respiratory rate">呼吸:18 次/分,特殊治療 :輸血中</Item>
<Item name="Respiratory rate">呼吸:18 次/分,特殊治療 :輸血前</Item>
<Item name="Systolic blood pressure">119</Item>
<Item name="Oral temperature">體溫:36.6 ℃,特殊治療 :輸血中</Item>
<Item name="Lactate [Mass/volume] in Serum or Plasma">13.8</Item>
<Item name="Lymphocytes [#/volume] in Blood">872</Item>
<Item name="Neutrophils [#/volume] in Blood">10965</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">8.7</Item>
<Item name="MCH [Entitic mass] by Automated count">31.6</Item>
<Item name="MCHC [Mass/volume] by Automated count">33.0</Item>
<Item name="Platelet mean volume [Entitic volume] in Blood by Automated count">10.7</Item>
<Item name="Erythrocyte distribution width [Ratio] by Automated count">16.3</Item>
<Item name="Leukocytes [#/volume] in Blood">12460</Item>
<Item name="Basophils/100 leukocytes in Blood">0.0</Item>
<Item name="Eosinophils/100 leukocytes in Blood">0.0</Item>
<Item name="Lymphocytes/100 leukocytes in Blood">7.0</Item>
<Item name="Monocytes/100 leukocytes in Blood">4.0</Item>
<Item name="Myelocytes/100 leukocytes in Blood by Manual count">1.0</Item>
<Item name="Nucleated erythrocytes/100 leukocytes [Ratio] in Blood">0.0</Item>
<Item name="Platelets [#/volume] in Blood by Automated count">162</Item>
<Item name="Segmented neutrophils/100 leukocytes in Blood">88.0</Item>
<Item name="aPTT in Platelet poor plasma by Coagulation assay">39.9</Item>
<Item name="Prothrombin time (PT) in Control Platelet poor plasma by Coagulation assay">10.5</Item>
<Item name="Prothrombin time (PT)">11.9</Item>
<Item name="INR in Platelet poor plasma by Coagulation assay">1.14</Item>
<Item name="A Ag [Presence] on Red Blood Cells">4</Item>
<Item name="A,B Ag [Presence] on Red Blood Cells">4</Item>
<Item name="B Ag [Presence] on Red Blood Cells">-</Item>
<Item name="Blood group antibody screen [Presence] in Serum or Plasma">NEG</Item>
<Item name="D Ag [Presence] on Red Blood Cells">4</Item>
<Item name="A Ab [Titer] in Serum or Plasma by Immediate spin">-</Item>
<Item name="B Ab [Titer] in Serum or Plasma by Immediate spin">4</Item>
<Item name="ABO and Rh group [Type] in Blood">A+</Item>
<Item name="Glucose [Mass/volume] in Serum or Plasma">345</Item>
<Item name="Base excess in Venous blood by calculation">2.9</Item>
<Item name="Bicarbonate [Moles/volume] in Venous blood">27.2</Item>
<Item name="Carbon dioxide [Partial pressure] in Venous blood">40.5</Item>
<Item name="pH of Venous blood">7.444</Item>
<Item name="Oxygen [Partial pressure] in Venous blood">69.9</Item>
<Item name="Oxygen saturation in Venous blood">92.9</Item>
<Item name="Carbon dioxide, total [Moles/volume] in Venous blood">28.4</Item>
<Item name="Albumin [Mass/volume] in Serum or Plasma">1.6</Item>
<Item name="Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma">238</Item>
<Item name="Diastolic blood pressure">55</Item>
<Item name="Diastolic blood pressure">85</Item>
<Item name="Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma by No addition of P-5'-P">14</Item>
<Item name="Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction">157</Item>
<Item name="Protein [Mass/volume] in Serum or Plasma">4.7</Item>
<Item name="Bilirubin.total [Mass/volume] in Serum or Plasma">0.94</Item>
<Item name="Ammonia [Mass/volume] in Plasma">73</Item>
<Item name="Urea nitrogen [Mass/volume] in Serum or Plasma">42</Item>
<Item name="Calcium [Mass/volume] in Serum or Plasma">8.3</Item>
<Item name="Oxygen saturation in Blood">95</Item>
<Item name="Creatinine [Mass/volume] in Serum or Plasma">5.37</Item>
<Item name="Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)">11.22</Item>
<Item name="Potassium [Moles/volume] in Serum or Plasma">3.1</Item>
<Item name="Sodium [Moles/volume] in Serum or Plasma">134</Item>
<Item name="Lipase [Enzymatic activity/volume] in Serum or Plasma">7</Item>
</LabReportGroup>
<NursingEvent timestamp="2023-04-10 08:00:00">
<VitalSign type="STOOL.STOOL" value="排便:1 次" />
</NursingEvent>
<NursingEvent timestamp="2023-04-10 20:00:00">
<SOAPNote>
<Intervention>醫師診視後，依醫囑設立靜脈輸液管路、抽血檢驗，(針)Desmopressin 4 AMP IVA Q12H,0.45% G/S 1000 ML IVD QD,Pantoprazole 1 VIAL IVA Q12H設床ED1-138後臥床休息。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-10 21:00:00">
<SOAPNote>
<Objective>X光顯示右側肺炎。</Objective><Intervention>醫囑予Tazocin inj 1 VIAL IVA Q8H首劑使用。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-10 22:00:00">
<SOAPNote>
<Objective>血液報告:HGB 8.7 g/dL、PT：11.9 SEC、APTT：39.9 SEC，預輸FFP 4U、PRBC 2U。</Objective><Intervention>填妥輸血同意書。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-10 22:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.8 ℃,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2023-04-10 22:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:118 次/分,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2023-04-10 22:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2023-04-10 22:00:00">
<VitalSign type="BP" value="血壓:119/55 mmHg,特殊治療 :輸血前" />
<SOAPNote>
<Intervention>依醫囑給予輸血前藥物Chlorpheniramine inj 1 AMP IV STAT，密切觀察有無輸血反應，予家屬衛教輸血注意事項衛教，若開始輸血前 15 分鐘有不適反應如發冷、發熱、呼吸困難、皮膚紅疹、皮膚癢、尿液變紅色等症狀，請告知護理師。</Intervention><Evaluation>家屬可了解衛教內容。</Evaluation>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-10 23:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.6 ℃,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2023-04-10 23:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:111 次/分,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2023-04-10 23:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2023-04-10 23:00:00">
<VitalSign type="BP" value="血壓:162/85 mmHg,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2023-04-10 23:00:00">
<VitalSign type="SPO2" value="SPO2:95.0 %" />
<SOAPNote>
<Objective>，暫無不適之主訴。</Objective>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-10 23:00:00">
<SOAPNote>
<Objective>GCS:E3V5M5，躁動不安、無法遵從醫療措施，出現自拔點滴。</Objective><Intervention>向病人或家屬詳細解釋約束之目的、方法及提供之照護，獲得同意並填寫「保護性身體約束同意書」，依醫囑予保護性身體約束，使用手腕式約束帶，予情緒安撫，衛教保護性身體約束相關之注意事項，家屬或看護需在旁陪伴。</Intervention><Evaluation>家屬表示能接受且知道保護性身體約束目的及注意事項，皮膚完整有無水腫、麻木感、潰瘍或出現新壓瘡發生，肢體末梢溫暖、膚色粉紅，無意外事件。</Evaluation>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-10 23:00:00">
<SOAPNote>
<Objective>血液報告:Albumin 1.6 g/dL。</Objective><Intervention>醫囑予25% Albunorm inj 50 ML IVD Q12H首劑使用。</Intervention>
</SOAPNote>
</NursingEvent>
<LabReportGroup date="2023-04-11">
<Item name="Bilirubin.total [Presence] in Urine by Automated test strip">Negative</Item>
<Item name="Clarity in Urine by Refractometry automated">Clear</Item>
<Item name="Color of Urine by Auto">Yellow</Item>
<Item name="Glucose [Mass/volume] in Urine by Automated test strip">250</Item>
<Item name="Ketones [Presence] in Urine by Automated test strip">Negative</Item>
<Item name="Leukocyte esterase [Presence] in Urine by Automated test strip">3+</Item>
<Item name="Nitrite [Presence] in Urine by Automated test strip">Positive</Item>
<Item name="Hemoglobin [Presence] in Urine by Automated test strip">2+</Item>
<Item name="pH of Urine by Automated test strip">6.0</Item>
<Item name="Protein [Presence] in Urine by Automated test strip">2+</Item>
<Item name="Specific gravity of Urine by Refractometry automated">1.015</Item>
<Item name="Urobilinogen [Units/volume] in Urine by Test strip">0.2</Item>
<Item name="Epithelial cells [#/area] in Urine sediment by Microscopy high power field">0-5</Item>
<Item name="Microscopic observation [Identifier] in Urine sediment by Light microscopy">Bacteria +</Item>
<Item name="Erythrocytes [#/area] in Urine sediment by Microscopy high power field">50-99</Item>
<Item name="Leukocytes [#/area] in Urine sediment by Microscopy high power field">>=100</Item>
<Item name="Hemoglobin.gastrointestinal [Presence] in Stool">Trace</Item>
<Item name="Ova and parasites identified in Stool by Light microscopy">Not Found</Item>
<Item name="Microscopic observation [Identifier] in Sputum by Gram stain">Smear contains > 25 Squamous epithelial cell / LPF,</Item>
<Item name="Leukocytes [#/area] in Stool by Microscopy high power field">-</Item>
<Item name="Diastolic blood pressure">80</Item>
<Item name="Diastolic blood pressure">60</Item>
<Item name="Diastolic blood pressure">77</Item>
<Item name="Diastolic blood pressure">67</Item>
<Item name="Diastolic blood pressure">47</Item>
<Item name="Diastolic blood pressure">73</Item>
<Item name="Diastolic blood pressure">62</Item>
<Item name="Diastolic blood pressure">66</Item>
<Item name="Oxygen saturation in Blood">91</Item>
<Item name="Oxygen saturation in Blood">98</Item>
<Item name="Oxygen saturation in Blood">96</Item>
<Item name="Oxygen saturation in Blood">95</Item>
<Item name="Systolic blood pressure">167</Item>
<Item name="Oxygen saturation in Blood">99</Item>
<Item name="Systolic blood pressure">115</Item>
<Item name="Systolic blood pressure">120</Item>
<Item name="Systolic blood pressure">130</Item>
<Item name="Systolic blood pressure">152</Item>
<Item name="Systolic blood pressure">158</Item>
<Item name="Systolic blood pressure">133</Item>
<Item name="Systolic blood pressure">145</Item>
<Item name="Glucose [Presence] in Blood by Test strip">血糖:196.0 mg/dl</Item>
<Item name="Glucose [Presence] in Blood by Test strip">血糖:110.0 mg/dl</Item>
<Item name="Glucose [Presence] in Blood by Test strip">血糖:347 mg/dl</Item>
<Item name="Heart rate">一般脈搏:120 次/分,特殊治療 :輸血後</Item>
<Item name="Heart rate">一般脈搏:136 次/分</Item>
<Item name="Heart rate">脈搏:122 次/分</Item>
<Item name="Heart rate">脈搏:118 次/分</Item>
<Item name="Heart rate">一般脈搏:114 次/分</Item>
<Item name="Heart rate">脈搏:125 次/分</Item>
<Item name="Heart rate">脈搏:122 次/分</Item>
<Item name="Heart rate">一般脈搏:114 次/分</Item>
<Item name="Heart rate">一般脈搏:110 次/分</Item>
<Item name="Heart rate">脈搏:122 次/分</Item>
<Item name="Heart rate">一般脈搏:128 次/分</Item>
<Item name="Oral temperature">體溫:37.1 ℃</Item>
<Item name="Oral temperature">體溫:37.2 ℃,特殊治療 :輸血後</Item>
<Item name="Oral temperature">體溫:37.2 ℃</Item>
<Item name="Oral temperature">體溫:36.6 ℃</Item>
<Item name="Respiratory rate">呼吸:18 次/分</Item>
<Item name="Respiratory rate">呼吸:20 次/分</Item>
<Item name="Respiratory rate">呼吸:20 次/分,特殊治療 :輸血後</Item>
<Item name="Respiratory rate">呼吸:20 次/分</Item>
<Item name="Respiratory rate">呼吸:18 次/分</Item>
<Item name="Respiratory rate">呼吸:15 次/分</Item>
<Item name="Respiratory rate">呼吸:15 次/分</Item>
<Item name="Respiratory rate">呼吸:15 次/分</Item>
<Item name="Respiratory rate">呼吸:15 次/分</Item>
<Item name="Respiratory rate">呼吸:15 次/分</Item>
<Item name="Respiratory rate">呼吸:18 次/分</Item>
<Item name="Microscopic observation [Identifier] in Sputum by Gram stain">Culture not performed. Please recollect if clinically indicated.</Item>
<Item name="Colony count [#] in Urine by Visual count">>100000</Item>
<Item name="Gentamicin [Susceptibility] by Minimum inhibitory concentration (MIC)">S</Item>
<Item name="Linezolid [Susceptibility] by Minimum inhibitory concentration (MIC)">S=2</Item>
<Item name="Penicillin G [Susceptibility] by Minimum inhibitory concentration (MIC)">R>=64</Item>
<Item name="Teicoplanin [Susceptibility] by Minimum inhibitory concentration (MIC)">R>=32</Item>
<Item name="Vancomycin [Susceptibility] by Minimum inhibitory concentration (MIC)">R>=32</Item>
<Item name="Bacteria identified in Urine by Culture">Enterococcus faecium</Item>
<Item name="Bacteria identified in Urine by Culture">Gram negative bacillus</Item>
<Item name="Amikacin [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=2</Item>
<Item name="cefTAZidime [Susceptibility] by Minimum inhibitory concentration (MIC)">S=0.5</Item>
<Item name="Cefoperazone+Sulbactam [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=8</Item>
<Item name="Tigecycline [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=0.5</Item>
<Item name="Ciprofloxacin [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=0.25</Item>
<Item name="cefTRIAXone [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=0.25</Item>
<Item name="Ertapenem [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=0.12</Item>
<Item name="Cefepime [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=0.12</Item>
<Item name="Flomoxef [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=2</Item>
<Item name="cefOXitin [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=4</Item>
<Item name="Gentamicin [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=1</Item>
<Item name="Imipenem [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=0.25</Item>
<Item name="ceFAZolin [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=4</Item>
<Item name="Ampicillin+Sulbactam [Susceptibility] by Minimum inhibitory concentration (MIC)">R>=32</Item>
<Item name="Trimethoprim+Sulfamethoxazole [Susceptibility] by Minimum inhibitory concentration (MIC)">R>=320</Item>
<Item name="Piperacillin+Tazobactam [Susceptibility] by Minimum inhibitory concentration (MIC)">S<=4</Item>
<Item name="Bacteria identified in Blood by Culture">Escherichia coli</Item>
</LabReportGroup>
<NursingEvent timestamp="2023-04-11 00:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:128 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 00:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:20 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 00:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.6 ℃" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 00:00:00">
<VitalSign type="SPO2" value="SPO2:91.0 %" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 00:00:00">
<SOAPNote>
<Objective>血糖347mg/dL。</Objective><Intervention>告知醫師。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 00:21:00">
<VitalSign type="GLUCOSE" value="血糖:347 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 01:00:00">
<VitalSign type="IS.4D" value="胰島素:部位 :左腹4D 藥物處置:Actrapid HM 1U 10 U SC STAT。" />
<SOAPNote>
<Objective>AT 0013血糖值:347mg/dL</Objective>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="BT.ORAL" value="體溫:37.2 ℃,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:120 次/分,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:20 次/分,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="BP" value="血壓:115/60 mmHg,特殊治療 :輸血後" />
<SOAPNote>
<Intervention>依醫囑給予(針)Rosis inj 20mg (80 MG) IVA ONCALL</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="COMASCALE.COMA.EYE" value="昏迷指標 E4V4M6" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="COMASCALE.COMA.TONGUE" value="昏迷指標 E4V4M6" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="COMASCALE.COMA.MOTION" value="昏迷指標 E4V4M6" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="COMASCALE.PUPIL.RIGHT" value="瞳孔指標 LEFT:－ , 2.5 , RIGHT:－ , 2.5" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 02:00:00">
<VitalSign type="COMASCALE.PUPIL.LEFT" value="瞳孔指標 LEFT:－ , 2.5 , RIGHT:－ , 2.5" />
<SOAPNote>
<Objective>，氧氣鼻導管3L/MIN使用下呼吸平順，右手周邊靜脈留置針存，注射帽使用，注射部位無紅腫滲漏，左手AV-SHUNT留置，洗腎用(W2、4、6)，禁治療手圈穿戴，現少尿，雙手現手腕式約束帶使用，身上無現存壓力性損傷，跌倒評估分數：1分，家屬伴。</Objective>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 05:00:00">
<VitalSign type="COMASCALE.COMA.EYE" value="昏迷指標 E4V4M6" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 05:00:00">
<VitalSign type="COMASCALE.COMA.TONGUE" value="昏迷指標 E4V4M6" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 05:00:00">
<VitalSign type="COMASCALE.COMA.MOTION" value="昏迷指標 E4V4M6" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 05:00:00">
<VitalSign type="COMASCALE.PUPIL.LEFT" value="瞳孔指標 LEFT:－ , 2.5 , RIGHT:－ , 2.5" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 05:00:00">
<VitalSign type="COMASCALE.PUPIL.RIGHT" value="瞳孔指標 LEFT:－ , 2.5 , RIGHT:－ , 2.5" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 05:00:00">
<VitalSign type="GLUCOSE" value="血糖:110.0 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 06:00:00">
<VitalSign type="INPUT.BLOOD" value="輸血量 380.0 CC" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 06:00:00">
<VitalSign type="INPUT.BLOOD" value="輸血量 660.0 CC" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 06:00:00">
<VitalSign type="OUTPUT.URINE" value="尿量 100 CC" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 07:00:00">
<SOAPNote>
<Objective>輸入量:1040cc; 輸出量:100cc; 差異: +940cc(23:01~07:00)。</Objective><Intervention>已告知黃上軒醫師，囑續觀察。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 07:00:00">
<SOAPNote>
<Subjective>家屬代訴平時常常嗆咳</Subjective><Objective>抽痰時甚多白色顆粒帶食物</Objective><Intervention>依醫囑予以放置16FR一般材質鼻胃管</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 07:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:20 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 07:00:00">
<VitalSign type="SPO2" value="SPO2:98.0 %" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 07:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:136 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 08:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:114 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 08:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 08:00:00">
<VitalSign type="SPO2" value="SPO2:96.0 %" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 08:00:00">
<VitalSign type="BP" value="血壓:120/77 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 08:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:114 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 08:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 08:00:00">
<VitalSign type="BT.ORAL" value="體溫:37.2 ℃" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 08:00:00">
<VitalSign type="SPO2" value="SPO2:95.0 %" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 10:00:00">
<SOAPNote>
<NarrativeNote>更換尿布及床單時，自拔NG，告知醫師</NarrativeNote>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 10:00:00">
<SOAPNote>
<Objective>預留尿液檢體</Objective><Intervention>依醫囑予協助醫師單導、並重新放置鼻胃管</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 11:00:00">
<SOAPNote>
<NarrativeNote>轉出摘要：目前問題（Situation）：目前病人因Malignant tumor of pancreas  ，經醫療團隊評估轉至：W-102-182病房，轉出時間：112年04月11日，繼續接受治療。背景資料（Background）：胰臟癌病史，此次因柏油樣大便解 2 天，今天早上嗜睡 入急診，X光顯示右側肺炎、血液報告:HGB 8.7 g/dL、PT：11.9 SEC、APTT：39.9 SEC、CT: Lung abscess at RLL.
，急診期間使用藥物(針)Desmopressin 4mcg (4 AMP) IVA Q12H, 25% Albunorm inj 50mL (50 ML) IVD Q12H, Half Saline 500cc (500 ML) IVD QD, Inj Pantoprazole (1 VIAL) IVA Q12H, Tazocin inj 2.25gm (1 VIAL) IVA Q8H。評估（Assessment）：意識狀態：E4V4M5，瞳孔2.5(+)，氧氣鼻導管3L/MIN使用，呼吸平順，右手周邊靜脈留置針存，鼻胃管灌食用，注射部位無紅腫滲漏，左手AV-SHUNT留置，洗腎用(W2、4、6)，禁治療手圈穿戴，現少尿，雙手現手腕式約束帶使用，身上無現存壓力性損傷，家屬伴。建議（Recommendation）：注意生命徵象監測。</NarrativeNote>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 12:00:00">
<SOAPNote>
<NarrativeNote>送洗腎室</NarrativeNote>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 12:00:00">
<VitalSign type="BT.ORAL" value="體溫:37.1 ℃" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 12:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:110 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 12:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 12:00:00">
<VitalSign type="BP" value="血壓:130/67 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 12:00:00">
<VitalSign type="SPO2" value="SPO2:99.0 %" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 13:00:00">
<SOAPNote>
<Objective>勤務和家人陪同推床入，呼吸平順，左手動靜脈廔管功能可，透析前血壓：152/47 mmHg。</Objective><Intervention>依醫囑行血液透析3小時30分，預脫水1.5-2.0 公升，每60分鐘生理食鹽水100mL沖洗管路，預防透析器血液凝固。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 13:00:00">
<VitalSign type="PULSE.PULSE" value="脈搏:122 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 13:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:15 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 13:00:00">
<VitalSign type="BP" value="血壓:152/47 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 13:00:00">
<VitalSign type="PULSE.PULSE" value="脈搏:118 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 13:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:15 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 13:00:00">
<VitalSign type="BP" value="血壓:158/73 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 14:00:00">
<VitalSign type="PULSE.PULSE" value="脈搏:125 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 14:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:15 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 14:00:00">
<VitalSign type="BP" value="血壓:133/62 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 14:00:00">
<VitalSign type="GLUCOSE" value="血糖:196.0 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 15:00:00">
<VitalSign type="PULSE.PULSE" value="脈搏:122 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 15:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:15 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 15:00:00">
<VitalSign type="BP" value="血壓:145/66 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 16:00:00">
<VitalSign type="PULSE.PULSE" value="脈搏:122 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 16:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:15 次/分" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 16:00:00">
<VitalSign type="BP" value="血壓:167/80 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2023-04-11 16:00:00">
<SOAPNote>
<Objective>at1631完成透析。</Objective><Intervention>結束透析。</Intervention><Evaluation>今共治療3小時30分脫水1.6公升，左手針眼以紗布及止血帶加壓止血中，血壓:167/80㎜Hg、心跳:122次/分，at1645由勤務人員協助下推床至102病房。</Evaluation><NarrativeNote>家屬已辦理好住院手續。</NarrativeNote>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2023-04-11 16:00:00">
<VitalSign type="OUTPUT.UF" value="洗腎脫水量 1600 CC" />
</NursingEvent>
    </ChronologicalEvents>
</PatientEncounter>"""
process_consultation(model, tokenizer, consultation_content, temperature=0.1, max_tokens=1024)
############# Sample Output #############
# In the ER, con's disturbance was noted. Lab data showed leukocytosis with elevated CRP. CXR showed right lung increased infiltration, 
# and lung CT dislcosed RLL lung abscess. CS was consulted, and conservative treatment was suggested. Stool OB was collected for tarry 
# stool, but trace OB was noted. EGD was not performed due to poor general condition. He is admitted to GI ward.

In [ ]:
consultation_content = """<PatientEncounter summary_length_style="short">
    <Summary>
        <PrimaryDiagnosis>UTI-(knocking(-), /p Flumarin);LLQ abdominal pain;</PrimaryDiagnosis>
<SecondaryDiagnosis>Anemia-(Hb:6.7->2U->7.0->2U);Submucous leiomyoma of uterus-(since 2021, Abdomen CT: Enhanced nodules at S7, S3;  Progressive myomas);Fever, source to be determined-(/p Flomoxef,PCRs(-));</SecondaryDiagnosis>
<PastMedicalHistory>Gall bladder stone;DM (Diabetes mellitus );</PastMedicalHistory>
<ChiefComplaint>LLQ pain, fever in recent 2 days
</ChiefComplaint>
<PresentIllness>42 y/o woman wiht a hx of type 2DM; chronic anemia; uterine myoma and adenomysis; no allergy hx; LMP 11/20; denies pregnancy; TOCC(covid in 2022; moderna2); She presents to ED because of LLQ pain, fever in recent 2 days. There is no URI symptoms, no headache, no focal weakness, no neck pain, no chest pain, no SOB, no dysuria, no cellulitis, no arthritis.
</PresentIllness>
    </Summary>
    <ChronologicalEvents>
        <NursingEvent timestamp="2024-11-20 08:00:00">
<VitalSign type="STOOL.STOOL" value="排便:0 次" />
</NursingEvent>
<LabReportGroup date="2024-11-21">
<Item name="Systolic blood pressure">101</Item>
<Item name="Systolic blood pressure">109</Item>
<Item name="Systolic blood pressure">99</Item>
<Item name="Systolic blood pressure">94</Item>
<Item name="Systolic blood pressure">107</Item>
<Item name="Systolic blood pressure">128</Item>
<Item name="Systolic blood pressure">102</Item>
<Item name="Systolic blood pressure">110</Item>
<Item name="Systolic blood pressure">108</Item>
<Item name="Nucleated erythrocytes/100 leukocytes [Ratio] in Blood">0.0</Item>
<Item name="A Ag [Presence] on Red Blood Cells">4</Item>
<Item name="Erythrocytes [#/area] in Urine sediment by Microscopy high power field">>=100</Item>
<Item name="Leukocytes [#/area] in Urine sediment by Microscopy high power field">>=100</Item>
<Item name="Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma">78</Item>
<Item name="Microscopic observation [Identifier] in Urine sediment by Light microscopy">Bacteria +</Item>
<Item name="Glucose [Mass/volume] in Urine by Automated test strip">100</Item>
<Item name="Platelet mean volume [Entitic volume] in Blood by Automated count">----</Item>
<Item name="Platelet mean volume [Entitic volume] in Blood by Automated count">----</Item>
<Item name="B Ab [Titer] in Serum or Plasma by Immediate spin">4</Item>
<Item name="A Ab [Titer] in Serum or Plasma by Immediate spin">-</Item>
<Item name="Ketones [Presence] in Urine by Automated test strip">3+</Item>
<Item name="Protein [Presence] in Urine by Automated test strip">3+</Item>
<Item name="Hemoglobin [Presence] in Urine by Automated test strip">3+</Item>
<Item name="Leukocyte esterase [Presence] in Urine by Automated test strip">1+</Item>
<Item name="SARS-CoV-2 (COVID-19) RNA [Presence] in Nasopharynx by NAA with probe detection">Negative (SARS-CoV-2 RNA was not detected)</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">6.7</Item>
<Item name="Hemoglobin [Mass/volume] in Blood">7.0</Item>
<Item name="Blood group antibody screen [Presence] in Serum or Plasma">NEG</Item>
<Item name="Lymphocytes/100 leukocytes in Blood">8.5</Item>
<Item name="Base excess in Venous blood by calculation">-0.5</Item>
<Item name="Iron [Mass/volume] in Serum or Plasma">14</Item>
<Item name="ABO and Rh group [Type] in Blood">A+</Item>
<Item name="Glucose [Mass/volume] in Serum or Plasma">175</Item>
<Item name="Lipase [Enzymatic activity/volume] in Serum or Plasma">11</Item>
<Item name="Casts [#/area] in Urine sediment by Microscopy low power field">Granular Cast 3-5</Item>
<Item name="B Ag [Presence] on Red Blood Cells">-</Item>
<Item name="A,B Ag [Presence] on Red Blood Cells">4</Item>
<Item name="Urobilinogen [Units/volume] in Urine by Test strip">1.0</Item>
<Item name="Eosinophils/100 leukocytes in Blood">0.8</Item>
<Item name="Calcium [Mass/volume] in Serum or Plasma">8.6</Item>
<Item name="Creatine kinase [Enzymatic activity/volume] in Serum or Plasma">18</Item>
<Item name="Choriogonadotropin.beta subunit (pregnancy test) [Presence] in Urine">Negative</Item>
<Item name="MCHC [Mass/volume] by Automated count">24.5</Item>
<Item name="MCHC [Mass/volume] by Automated count">26.1</Item>
<Item name="Carbon dioxide, total [Moles/volume] in Venous blood">23.2</Item>
<Item name="C reactive protein [Mass/volume] in Serum or Plasma by High sensitivity method">5.421</Item>
<Item name="Epithelial cells [#/area] in Urine sediment by Microscopy high power field">0-5</Item>
<Item name="Leukocytes [#/volume] in Blood">20420</Item>
<Item name="MCH [Entitic mass] by Automated count">14.7</Item>
<Item name="MCH [Entitic mass] by Automated count">16.8</Item>
<Item name="pH of Venous blood">7.412</Item>
<Item name="Urea nitrogen [Mass/volume] in Serum or Plasma">10</Item>
<Item name="Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma">30</Item>
<Item name="Creatinine [Mass/volume] in Serum or Plasma">0.46</Item>
<Item name="D Ag [Presence] on Red Blood Cells">4</Item>
<Item name="Neutrophils [#/volume] in Blood">17520</Item>
<Item name="Neutrophils/100 leukocytes in Blood">85.8</Item>
<Item name="aPTT in Platelet poor plasma by Coagulation assay">26.9</Item>
<Item name="Monocytes/100 leukocytes in Blood">4.8</Item>
<Item name="Lymphocytes [#/volume] in Blood">1736</Item>
<Item name="Sodium [Moles/volume] in Serum or Plasma">140</Item>
<Item name="Erythrocyte distribution width [Ratio] by Automated count">20.0</Item>
<Item name="Erythrocyte distribution width [Ratio] by Automated count">24.5</Item>
<Item name="Carbon dioxide [Partial pressure] in Venous blood">38.6</Item>
<Item name="Basophils/100 leukocytes in Blood">0.1</Item>
<Item name="INR in Platelet poor plasma by Coagulation assay">1.01</Item>
<Item name="Potassium [Moles/volume] in Serum or Plasma">3.4</Item>
<Item name="Bacteria identified in Blood by Culture">No bacterial growth for 7 days.</Item>
<Item name="MCV [Entitic volume] by Automated count">59.9</Item>
<Item name="Bilirubin.total [Mass/volume] in Serum or Plasma">0.64</Item>
<Item name="Oxygen [Partial pressure] in Venous blood">49.7</Item>
<Item name="Albumin [Mass/volume] in Serum or Plasma">4.4</Item>
<Item name="Oxygen saturation in Venous blood">87.4</Item>
<Item name="Platelets [#/volume] in Blood by Automated count">243</Item>
<Item name="Bacteria identified in Urine by Culture">No growth for ordinary culture.</Item>
<Item name="Bicarbonate [Moles/volume] in Venous blood">24.0</Item>
<Item name="Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma by No addition of P-5'-P">71</Item>
<Item name="pH of Urine by Automated test strip">5.0</Item>
<Item name="Clarity in Urine by Refractometry automated">Turbid</Item>
<Item name="Color of Urine by Auto">Red</Item>
<Item name="Specific gravity of Urine by Refractometry automated">1.036</Item>
<Item name="Bilirubin.total [Presence] in Urine by Automated test strip">1+</Item>
<Item name="Nitrite [Presence] in Urine by Automated test strip">Positive</Item>
<Item name="Prothrombin time (PT) in Control Platelet poor plasma by Coagulation assay">10.7</Item>
<Item name="Prothrombin time (PT)">10.8</Item>
<Item name="Influenza virus A RNA [Presence] in Nasopharynx by NAA with probe detection">NEGATIVE</Item>
<Item name="Influenza virus B RNA [Presence] in Nasopharynx by NAA with probe detection">NEGATIVE</Item>
<Item name="Respiratory syncytial virus RNA [Presence] in Nasopharynx by NAA with probe detection">NEGATIVE</Item>
<Item name="Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)">148.97</Item>
<Item name="Diastolic blood pressure">57</Item>
<Item name="Diastolic blood pressure">59</Item>
<Item name="Diastolic blood pressure">54</Item>
<Item name="Diastolic blood pressure">50</Item>
<Item name="Diastolic blood pressure">63</Item>
<Item name="Diastolic blood pressure">78</Item>
<Item name="Diastolic blood pressure">67</Item>
<Item name="Diastolic blood pressure">56</Item>
<Item name="Diastolic blood pressure">59</Item>
</LabReportGroup>
<NursingEvent timestamp="2024-11-21 01:00:00">
<SOAPNote>
<Intervention>醫師診視後，依醫囑設立靜脈輸液管路、抽血檢驗，TramaL (50 MG) IVA STAT,南光 0.9% Saline  (1000 ML) IVD QD設床ED1-112後臥床休息。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 01:00:00">
<SOAPNote>
<Objective>意識清，呼吸順，周邊靜脈導管滴注暢，注射部位無紅腫滲液，無壓力性損傷，跌倒評估為2分，非屬高危險跌倒群，雙側床欄使用，臥床休息，家屬陪</Objective><Intervention>加強探視、注意腹痛情形及體溫變化、管路固定及活動安全。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 02:00:00">
<SOAPNote>
<Objective>抽血報告： WBC 20420 /μL、hsCRP: 5.421 mg/dL，經醫師評估</Objective><Intervention>依醫囑予首劑Flumarin (1000 MG) IVA Q8H使用。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 03:00:00">
<VitalSign type="PAIN" value="疼痛:8分" />
<SOAPNote>
<Subjective>肚子會痛</Subjective><Objective>，醫師前來診視</Objective><Intervention>依醫囑予Mutonpain (0.5 AMP) IVA STAT使用。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<SOAPNote>
<Objective>抽血追蹤HGB：6.7g/dL，醫師已知囑備輸LEUKOCYTE POOR RBC 2 U</Objective><Intervention>依醫囑輸血前給予Chlorpheniramin  1Amp  IVA 使用，經電子條碼核對病人資料及血型、血袋資料無誤，開始輸血，持續觀察有無輸血不良過敏反應。</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<VitalSign type="BT.ORAL" value="體溫:37.8 ℃,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:86 次/分,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<VitalSign type="BP" value="血壓:101/57 mmHg,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<VitalSign type="BT.ORAL" value="體溫:37.0 ℃,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:83 次/分,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 04:00:00">
<VitalSign type="BP" value="血壓:109/59 mmHg,特殊治療 :輸血中" />
<SOAPNote>
<Objective>輸血15分鐘後</Objective><Intervention>觀察生命徵象及有無輸血不良過敏反應，調整輸血速率，持續輸血中</Intervention><Evaluation>，無輸血不良過敏反應。</Evaluation>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 07:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.6 ℃,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 07:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:81 次/分,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 07:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 07:00:00">
<VitalSign type="BP" value="血壓:99/54 mmHg,特殊治療 :輸血後" />
<SOAPNote>
<Intervention>觀察生命徵象及有無輸血不良過敏反應</Intervention><Evaluation>，無輸血不良過敏反應。</Evaluation>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 08:00:00">
<VitalSign type="BP" value="血壓:94/50 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 08:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:90 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 08:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 08:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.5 ℃" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 08:00:00">
<VitalSign type="SPO2" value="SPO2:100.0 %" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 08:00:00">
<VitalSign type="GLUCOSE" value="血糖:171.0 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 11:00:00">
<VitalSign type="OUTPUT.BLOOD_LOSE" value="失血量 124 CC" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 11:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.7 ℃" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 11:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:88 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 11:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 11:00:00">
<VitalSign type="BP" value="血壓:107/63 mmHg" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 11:00:00">
<VitalSign type="SPO2" value="SPO2:97.0 %" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 12:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.9 ℃" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 12:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:97 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 12:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 12:00:00">
<VitalSign type="SPO2" value="SPO2:98.0 %" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 12:00:00">
<VitalSign type="BP" value="血壓:128/78 mmHg" />
<SOAPNote>
<Subjective>腹痛</Subjective><Objective>送婦產科</Objective><Intervention>醫囑給TramaL inj 100mg/2mL (50 MG) IVA Q8HPRN</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<SOAPNote>
<Objective>HGB 7.0 g/dL</Objective><Intervention>協助填妥輸血同意書，依醫囑給予Chlorpheniramine inj 5mg/1mL (1 AMP) IVD STAT, 南光 N.S. inj 250mL (250 ML) IVD STAT藥物, 輸PRBC2U，密切觀察有無輸血反應，予病人/家屬衛教輸血注意事項衛教，若開始輸血前 15 分鐘有不適反應如發冷、發熱、呼吸困難、皮膚紅疹、皮膚癢、尿 液變紅色等症狀，請告知護理師</Intervention><Evaluation>病人可了解衛教內容</Evaluation>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<VitalSign type="BT.ORAL" value="體溫:37.1 ℃,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:92 次/分,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<VitalSign type="BP" value="血壓:102/67 mmHg,特殊治療 :輸血前" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<VitalSign type="BT.ORAL" value="體溫:36.9 ℃,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:96 次/分,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<VitalSign type="BP" value="血壓:110/56 mmHg,特殊治療 :輸血中" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:00:00">
<SOAPNote>
<Subjective>腹痛，陰道出血</Subjective><Intervention>醫囑給Genuproxin tab 250mg (1 TAB) PO BIDPC, Tranexamic acid inj 250mg/5mL (1 AMP) IVA Q8H, Ulstop f.c. tab 20mg (1 TAB) PO BID</Intervention>
</SOAPNote>
</NursingEvent>
<NursingEvent timestamp="2024-11-21 14:08:00">
<VitalSign type="GLUCOSE" value="血糖:190 mg/dl" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 15:00:00">
<VitalSign type="OUTPUT.BLOOD_LOSE" value="失血量 74 CC" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 16:00:00">
<VitalSign type="BT.ORAL" value="體溫:37.7 ℃,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 16:00:00">
<VitalSign type="PULSE.USUAL" value="一般脈搏:92 次/分,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 16:00:00">
<VitalSign type="RESPIRATORY" value="呼吸:18 次/分,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 16:00:00">
<VitalSign type="BP" value="血壓:108/59 mmHg,特殊治療 :輸血後" />
</NursingEvent>
<NursingEvent timestamp="2024-11-21 18:00:00">
<SOAPNote>
<NarrativeNote>轉出摘要：目前問題（Situation）：目前病人因UTI，經醫療團隊評估轉至：86病房212床， 轉出時間：113年11月21日 ，繼續接受治療。背景資料（Background）：病人本身子宮肌腺症，子宮肌瘤病史，此次陰道出血入急診，急診抽血報告:hsCRP: 5.421 mg/dL WBC 20420 /μL ANC 17520 /uLK 3.4 mEq/LHGB：6.7g/dL-->HGB：7.0g/dL，腹部CT:注意到來自子宮的多個大腫瘤塊。首先考慮肌瘤，懷疑有變性病灶。與先前的CT相比，病灶顯示出進展。建議GYN進一步檢查。 右骨盆和卵巢靜脈擴張。懷疑骨盆腔充血綜合症。尿液檢體:O.B. 3+ ，三合一鼻篩檢皆陰性，急診期間使用藥物Flumarin inj 1Gm (1000 MG) IVA Q8H, Genuproxin tab 250mg (1 TAB) PO BIDPC, Tranexamic acid inj 250mg/5mL (1 AMP) IVA Q8H, Ulstop f.c. tab 20mg (1 TAB) PO BID評估（Assessment）：意識清，呼吸順，周邊點滴滴住順，注射部位無紅腫滲濕，臥床休息，自己一人建議（Recommendation）：注意生命徵象。</NarrativeNote>
</SOAPNote>
</NursingEvent>
    </ChronologicalEvents>
</PatientEncounter>"""
process_consultation(model, tokenizer, consultation_content, temperature=0.1, max_tokens=1024)
############# Sample Output #############
# At ED, massive menstrual bleeding was noted with anemia. Blood transfusion with pRBC 2U + 2U were administered. 
# Antibiotics with Flomoxef was given for UTI. Analgesics with NSAID combined with Transamin use were prescribed. 
# We had discussed with the patient about keep Gestrinone use for control of the adenomyosis and myoma, 
# and she agreed. She was admitted for antibiotics treatment and further management.

In [ ]:
##################
# Save the model #
##################

In [ ]:
# [NOTE] This ONLY saves the LoRA adapters, and not the full model. To save to GGUF, scroll down!
model.save_pretrained(new_model_name)  # Local saving
tokenizer.save_pretrained(new_model_name)

In [ ]:
model.save_pretrained_merged(new_model_name, tokenizer)

In [ ]:
model.save_pretrained_gguf(
    new_model_name,
    quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
)